In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import random
df_sp = pd.read_csv(r'F:\Abhilah_First\Results\Observed_T\noised_stake_data.csv')
df_sp.head()

,Unnamed: 0,Julian Day,obs_period(days),elevation(m),Derbis thickness (cm),melt (cm),pdd,Model-1 Melt Rate (cm/day),Model-2 Melt Rate (cm/day),Model-3 Melt Rate (cm/day)
0,0,232,17,3868.0,110.112269,9.226452,186.225022,0.975783,0.530753,0.657459
1,1,250,18,3867.0,113.032401,9.876500,155.421463,0.760865,0.408535,0.509296
2,2,263,13,3867.0,114.128684,1.758438,110.102815,0.743350,0.397225,0.496367
3,3,273,10,3868.0,116.278306,3.607321,79.189979,0.689702,0.365156,0.458390
4,4,298,25,3868.0,112.753976,0.653328,128.831619,0.454563,0.244369,0.304456


In [2]:
df_model1_param = pd.read_csv(r'F:\Abhilah_First\Results\Observed_T\Parameters_1000.csv')
df_model1_param.head()

,Unnamed: 0,TF1 Model 1,TF2 Model 1,MSE Model 1,DDF Model 2,d0 Model 2,MSE Model 2,DDF Model 3,d0 Model 3,a Model 3,MSE Model 3
0,0,1.377083,-0.193208,0.628261,4.882812,12.876337,0.427875,4.344631,6.342217,0.352784,0.427687
1,1,1.029540,-0.380523,0.534095,4.590246,15.585462,0.478000,3.862951,10.103826,0.408360,0.473450
2,2,0.974426,-0.384808,0.469312,4.490768,14.961118,0.422020,3.831872,8.800334,0.382067,0.417324
3,3,1.065527,-0.355470,0.513937,4.848621,13.643871,0.424900,4.073580,8.720120,0.401118,0.421400
4,4,1.029322,-0.359518,0.516735,4.687397,13.981979,0.442138,4.059110,8.195743,0.390163,0.436305


In [3]:
data = {'DDF':df_model1_param['DDF Model 2']} 
df_all_iterations = pd.DataFrame(data)
df_all_iterations['d0'] = df_model1_param['d0 Model 2']
MSEs = list(df_model1_param['MSE Model 2'])
df_all_iterations['MSE'] = df_model1_param['MSE Model 2']
weight = []
for i in MSEs:
    weight.append(math.exp(-i))
df_all_iterations['Weight'] = weight
df_all_iterations.head()

,DDF,d0,MSE,Weight
0,4.882812,12.876337,0.427875,0.651893
1,4.590246,15.585462,0.478000,0.620023
2,4.490768,14.961118,0.422020,0.655721
3,4.848621,13.643871,0.424900,0.653835
4,4.687397,13.981979,0.442138,0.642661


In [4]:
df_all_iterations['DDF*Weight'] = df_all_iterations['DDF']*df_all_iterations['Weight']
df_all_iterations['d0*Weight'] = df_all_iterations['d0']*df_all_iterations['Weight']

df_all_iterations['Weight*(DDF - xBARw)^2'] = df_all_iterations['Weight']*(df_all_iterations['DDF'] - sum(df_all_iterations['DDF*Weight']/1000) )**2
df_all_iterations['Weight*(d0 - yBARw)^2'] = df_all_iterations['Weight']*(df_all_iterations['d0'] - sum(df_all_iterations['d0*Weight']/1000) )**2

df_all_iterations.head()

,DDF,d0,MSE,Weight,DDF*Weight,d0*Weight,Weight*(DDF - xBARw)^2,Weight*(d0 - yBARw)^2
0,4.882812,12.876337,0.427875,0.651893,3.183071,8.393994,2.049102,10.014974
1,4.590246,15.585462,0.478000,0.620023,2.846056,9.663337,1.358780,27.243419
2,4.490768,14.961118,0.422020,0.655721,2.944690,9.810314,1.250373,23.640074
3,4.848621,13.643871,0.424900,0.653835,3.170200,8.920845,1.976701,14.363981
4,4.687397,13.981979,0.442138,0.642661,3.012406,8.985668,1.599312,16.228857


In [5]:
ddf_variability = (sum(df_all_iterations['Weight*(DDF - xBARw)^2'])*1000/(1000*(1000-1)*sum(df_all_iterations['Weight'])) )**(0.5)


d0_variability = (sum(df_all_iterations['Weight*(d0 - yBARw)^2'])*1000/(1000*(1000-1)*sum(df_all_iterations['Weight'])) )**(0.5)

print(df_all_iterations['DDF'][MSEs.index(min(MSEs))], df_all_iterations['d0'][MSEs.index(min(MSEs))])
print(round(ddf_variability,2), round(d0_variability,2))
print('RMSE =',min(MSEs)**0.5)

5.0458074582366175 12.762741292679125
0.05 0.16
RMSE = 0.57905425933483
